# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[num_test/2:], newsgroups_test.target[num_test/2:]
dev_data, dev_labels = newsgroups_test.data[:num_test/2], newsgroups_test.target[:num_test/2]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print 'training label shape:', train_labels.shape
print 'test label shape:', test_labels.shape
print 'dev label shape:', dev_labels.shape
print 'labels names:', newsgroups_train.target_names

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

[2 pts]

In [3]:
def P1(num_examples=5):
### STUDENT START ###
    for index in range(num_examples):
        print 'label=', newsgroups_train.target_names[train_labels[index]]
        print 'text=', train_data[index]
        print '\n-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*\n'

### STUDENT END ###
P1(5)

label= comp.graphics
text= Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

label= talk.religion.misc
text= 

Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh hav

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

[6 pts]

In [4]:
def P2():
### STUDENT START ###
    vectorizer = CountVectorizer()
    vectorizer_train = vectorizer.fit_transform(train_data)
    
    ## Part A...
    print 'a. size of vocabulary is={}'.format(vectorizer_train.shape[1])
    print 'a. average number of non-zero features per example is={}'.format(np.average(vectorizer_train.getnnz(axis=1)))
    
    ## Part B...
    print 'b. 0th feature string={}'.format(vectorizer.get_feature_names()[0])
    print 'b. last feature string={}'.format(vectorizer.get_feature_names()[-1])
    
    ## Part C...
    
    vocab = ["atheism", "graphics", "space", "religion"]
    vectorizer_with_vocab = CountVectorizer(vocabulary=vocab)
    vectorizer_vocab_train = vectorizer_with_vocab.fit_transform(train_data)
    print 'c. shape is={}'.format(vectorizer_vocab_train.shape)
    print 'c. average number of non-zero features per example is={}'.format(np.average(vectorizer_vocab_train.getnnz(axis=1)))
    
    ## Part D...
    
    vectorizer = CountVectorizer(analyzer="char", ngram_range=(2,3))
    doc_matrix = vectorizer.fit_transform(train_data)
    print 'd. size of vocabulary is={}'.format(doc_matrix.shape[1])
    
    ## Part E...
    
    vectorizer = CountVectorizer(min_df=10)
    doc_matrix = vectorizer.fit_transform(train_data)
    print 'e. size of vocabulary is={}'.format(doc_matrix.shape[1])
    
    ## Part F...
    
    vectorizer = CountVectorizer()
    vectorizer_train = vectorizer.fit_transform(train_data)
    features_train = vectorizer.get_feature_names()
    
    vectorizer_dev = vectorizer.fit_transform(dev_data)
    features_dev = vectorizer.get_feature_names()
    
    print 'f. vectorizer_train size={}'.format(vectorizer_train.shape[1])
    print 'f. vectorizer_dev size={}'.format(vectorizer_dev.shape[1])
    difference = set(features_dev).difference(features_train)
    print 'f. vocabulary size difference={}'.format(len(difference))
    
### STUDENT END ###
P2()

a. size of vocabulary is=26879
a. average number of non-zero features per example is=96.7059980334
b. 0th feature string=00
b. last feature string=zyxel
c. shape is=(2034, 4)
c. average number of non-zero features per example is=0.268436578171
d. size of vocabulary is=35478
e. size of vocabulary is=3064
f. vectorizer_train size=26879
f. vectorizer_dev size=16246
f. vocabulary size difference=4027


(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

[4 pts]

In [41]:
def P3():
### STUDENT START ###
    vectorizer1 = CountVectorizer()
    vc1_data = vectorizer1.fit_transform(train_data)
    
    vectorizer2 = CountVectorizer()
    vc2_data = vectorizer2.fit(train_data)
    vc2_data = vectorizer2.transform(dev_data)
    
    #Fit KNN Model..
    k_values = {'n_neighbors' : [n for n in range(1, 10)]}
    knn = KNeighborsClassifier()
    knn_model = GridSearchCV(knn, k_values, scoring="f1_macro")
    knn_model.fit(vc1_data, train_labels)
    score = knn_model.score(vc2_data, dev_labels) 
    
    print 'optimal k value is={}, with score={:.3f}'.format(knn_model.best_params_["n_neighbors"], score)
   
    ##Fit Multinomial Model...
    alphas = {'alpha' : [a/10. for a in range(0, 11)]}
    mm = MultinomialNB()
    mm_model = GridSearchCV(mm, alphas)
    mm_model.fit(vc1_data, train_labels)
    score = mm_model.score(vc2_data, dev_labels) 
    
    print 'multinomial optimal value is={}, with score={:.3f}'.format(mm_model.best_params_["alpha"], score)

    ##Fit Logistic Regression Model..
    
    c_values = {'C' : [a/10. for a in range(1, 11)]}
    log_reg = LogisticRegression()
    lr_model = GridSearchCV(log_reg, c_values)
    lr_model.fit(vc1_data, train_labels)
    score = lr_model.score(vc2_data, dev_labels) 
    
    print 'logistic regression optimal value is={}, with score={:.3f}'.format(lr_model.best_params_["C"], score)
    
### STUDENT END ###
P3()

optimal k value is=9, with score=0.423
multinomial optimal value is=0.1, with score=0.793
logistic regression optimal value is=0.2, with score=0.713


a. In general knn works poorly in high dimensions problems. This problem is an example of that. 

b. 

c. 

ANSWER:

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

[5 pts]

In [100]:
def P4(range):
### STUDENT START ###

    # create CountVectorizer with ngram_range 
    vectorizer = CountVectorizer(ngram_range=range)
    doc_matrix_train = vectorizer.fit_transform(train_data)
    doc_matrix_dev = vectorizer.transform(dev_data)
    
    # create and fit a logistic regression model...
    model = LogisticRegression()
    model.fit(doc_matrix_train, train_labels)
    
    # Determine 5 features with biggest weights for each of the 4 classes
    weights_sorted = np.argsort(model.coef_, axis=1)
    
    # Obtain index of maximum weights for each class, then flatten to create array of 20 features
    weights_maxindex = weights_sorted[:, -5:].flatten()    
    
    # draw the table..
    full_vocab = np.array(vectorizer.get_feature_names())
    print "vocabulary size={}".format(len(full_vocab))
    print "accuracy={}".format(model.score(doc_matrix_dev, dev_labels))
    print np.transpose(np.round(model.coef_[:, weights_maxindex], 2))

### STUDENT END ###
print 'Unigram (ngram_range=(1, 1))'
P4((1,1))

print '\n-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*\n' 

print 'Bigram (ngram_range=(2 ,2))'
P4((2,2))

Unigram (ngram_range=(1, 1))
vocabulary size=26879
accuracy=0.699704142012
[[ 0.94 -0.41 -0.45 -0.43]
 [ 0.95 -0.62 -0.79 -0.06]
 [ 0.99 -0.22 -0.34 -0.46]
 [ 1.03 -0.1  -0.32 -0.83]
 [ 1.12 -0.4  -0.42 -0.4 ]
 [ 0.14  0.98 -0.68 -0.49]
 [-0.36  1.12 -0.7  -0.38]
 [-0.33  1.27 -0.81 -0.63]
 [-0.58  1.35 -0.83 -0.47]
 [-0.76  1.94 -1.34 -0.76]
 [-0.36 -0.39  0.92 -0.38]
 [-0.47 -0.47  0.94 -0.33]
 [-0.57 -0.48  1.01 -0.47]
 [-0.41 -0.67  1.22 -0.63]
 [-1.26 -1.32  2.16 -1.17]
 [-0.79 -0.08 -0.15  0.91]
 [-0.31 -0.27 -0.45  0.91]
 [-0.53 -0.11 -0.32  1.05]
 [-0.61 -0.42 -0.27  1.12]
 [-0.74 -0.41 -0.52  1.15]]

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

Bigram (ngram_range=(2 ,2))
vocabulary size=194891
accuracy=0.627218934911
[[ 0.57 -0.32 -0.58  0.  ]
 [ 0.63 -0.24 -0.2  -0.17]
 [ 0.65 -0.88 -0.82  0.6 ]
 [ 0.68 -0.19 -0.2  -0.3 ]
 [ 0.77 -0.26 -0.35 -0.2 ]
 [-0.32  0.9  -0.58 -0.33]
 [-0.43  0.91 -0.57 -0.34]
 [-0.55  0.97 -0.53 -0.51]
 [-0.38  1.04 -0.47 -0.4 ]
 [-0.76

ANSWER:

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

[4 pts]

In [80]:
def empty_preprocessor(s):
    return s

def better_preprocessor(s):
    new_string = [x.lower() for x in s]
    new_string = [x.strip() for x in s]
    #new_string = [re.sub('[^A-Za-z]', '', x) for x in s]

    return new_string

def P5():
### STUDENT START ###
    train_processed_data = better_preprocessor(train_data)
    
    # no processing applied...
    vectorizer1 = CountVectorizer()
    vc1_data = vectorizer1.fit_transform(train_data)
    
    # processed applied...
    vectorizer2 = CountVectorizer()
    vc2_data = vectorizer2.fit_transform(train_processed_data)
    
    # Vectorize the dev data for scoring our logistic regression models
    vectorizer3 = CountVectorizer()
    vc3_data = vectorizer3.fit(train_data)
    vc3_data = vectorizer3.transform(dev_data)
    
    # Logistic regression without pre-processing
    lg_model_1 = LogisticRegression()
    lg_model_1.fit(vc1_data, train_labels)
    preds1 = lg_model_1.predict(vc3_data)
    score1 = metrics.f1_score(dev_labels, preds1, average="macro")
    print 'score of logistic regression model [before] processing is={:.4f}'.format(score1)
    
    # Logistic regression with pre-processing
    lg_model_2 = LogisticRegression()
    lg_model_2.fit(vc2_data, train_labels)    
    preds2 = lg_model_2.predict(vc3_data)    
    score2 = metrics.f1_score(dev_labels, preds2, average="macro")    
    print 'score of logistic regression model [after] processing is={:.4f}'.format(score2)

    
### STUDENT END ###
P5()

score of logistic regression model [before] processing is=0.6793
score of logistic regression model [after] processing is=0.6793


(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

[4 pts]

In [8]:
def P6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    ### STUDENT START ###

    

    ### STUDENT END ###
P6()

(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

[4 pts]

In [11]:
#def P7():
    ### STUDENT START ###



    ### STUDENT END ###
#P7()

ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.

- [1 pt] for a reasonable attempt
- [2 pts] for improved performance